# Reddit Data Collection and Analysis with PSAW

To collect Reddit data, we're going to use the [Pushift API](https://www.reddit.com/r/pushshift/comments/bcxguf/new_to_pushshift_read_this_faq/), specifically a Python wrapper for the Pushshift API called [PSAW](https://github.com/dmarx/psaw) (PushShift API Wrapper). Why are we using the Pushshift API instead of the official Reddit API, and PSAW instead of Pushshift itself?

Well, as Pushshift's creator Jason Baumgartner and his co-authors describe it in their [published paper](https://arxiv.org/pdf/2001.08435.pdf), "Pushshift makes it
much easier for researchers to query and retrieve historical Reddit data, provides extended functionality by providing fulltext search against comments and submissions, and has larger single query limits." PSAW, meanwhile, makes it easier to work with Pushshift and provides better documentation.

## Install PSAW

To use PSAW, we first need to install it.

In [ ]:
!pip install psaw

Then we will import pandas for eventually working with the collected data, and we will change pandas default display setting to make our DataFrame columns wider.

In [2]:
import pandas as pd
pd.set_option('max_colwidth', 500)
pd.set_option('max_columns', 50)

Next we will import the PushshiftAPI from psaw and initialize it.

In [3]:
from psaw import PushshiftAPI

# Initialize PushShift
api = PushshiftAPI()

## Collect Reddit Posts (By Subreddit)

To collect Reddit posts, we will use `api.search_submissions()` and then establish the parameters of our query, such as which subreddit we want to search in and what threshold of upvote score we want to set.

Below we are setting up to search for posts in the subreddit "AmITheAsshole" that have an upvote score of at least 2,000 or more.

In [5]:
api_request_generator = api.search_submissions(subreddit='AmITheAsshole',
                                               score = ">2000")

Once this generator is set up, we can use it to collect Reddit posts. The code below is a list comprehension that loops through the generator and extracts relevant data for each matching Reddit post. It then turns that list into a Pandas DataFrame.

<div class="admonition pandasreview" name="html-admonition" style="background: black; color: white; padding: 10px">
<p class="title">Pandas</p>
 Do you need a refresher or introduction to the Python data analysis library Pandas? Be sure to check out <a href="https://melaniewalsh.github.io/Intro-Cultural-Analytics/Data-Analysis/Pandas-Basics-Part1.html"> Pandas Basics (1-3) </a> in this textbook!
    
</div>

In [ ]:
aita_submissions = pd.DataFrame([submission.d_ for submission in api_request_generator])

*The cell above should take a while to run. It's searching through Reddit's entire history. It's ok if you periodically get errors while it's running.*

Let's check to see how many Reddit posts we have collected by checking the shape of the DataFrame.

In [7]:
aita_submissions.shape

(2959, 78)

We have 2,959 posts!

Let's check to see what columns/metadata exist in this data by seeing what columns are in the DataFrame.

In [8]:
aita_submissions.columns

Index(['all_awardings', 'allow_live_comments', 'author',
       'author_flair_css_class', 'author_flair_richtext', 'author_flair_text',
       'author_flair_type', 'author_fullname', 'author_patreon_flair',
       'author_premium', 'awarders', 'can_mod_post', 'contest_mode',
       'created_utc', 'domain', 'edited', 'full_link', 'gildings', 'id',
       'is_crosspostable', 'is_meta', 'is_original_content',
       'is_reddit_media_domain', 'is_robot_indexable', 'is_self', 'is_video',
       'link_flair_background_color', 'link_flair_richtext',
       'link_flair_text_color', 'link_flair_type', 'locked', 'media_only',
       'no_follow', 'num_comments', 'num_crossposts', 'over_18',
       'parent_whitelist_status', 'permalink', 'pinned', 'pwls',
       'retrieved_on', 'score', 'selftext', 'send_replies', 'spoiler',
       'stickied', 'subreddit', 'subreddit_id', 'subreddit_subscribers',
       'subreddit_type', 'suggested_sort', 'thumbnail', 'title',
       'total_awards_received', 'trea

To get a quick peak of the data, we can look at 10 random rows of data, and only for the columns "title" and upvote "score."

In [11]:
aita_submissions[['title', 'score']].sample(10)

title  \
1787                                                             AITA for not paying for college for my pregnant daughter?   
1563  AITA for offering my sister to pay for an abortion but not offering to support the child finacially if she keeps it?   
1422                  AITA for snapping at my sister when she encouraged her 4 y/o daughter to fingerprint with my makeup?   
2908                       AITA For purposely stopping my classmate from winning an award and subsequently making her cry?   
2412               AITA for not giving up my table (at a restaurant) for a pregnant woman who needed it for accessibility?   
2672    AITA for not allowing people to access a swimming hole that has been used for generations, but is on our property?   
1220                                           AITA for giving both of my kids the same money for Back to School Shopping?   
1707                                                       WIBTA if I stopped inviting my triggered friend to movie night?   
949                                                          AITA for leaving my girlfriend alone in the ER to go workout?   
566                    AITA for doing the bare minimum babysitting because my mom and stepdad expect me to fo it for free?   

      score  
1787   5913  
1563   9111  
1422   4420  
2908  32356  
2412  11900  
2672   4618  
1220   3749  
1707  12656  
949    2718  
566    4467

To sort by and manipulate date information, let's transform the date into datetime values.

In [8]:
aita_submissions['date'] = pd.to_datetime(aita_submissions['created_utc'], utc=True, unit='s')

To only look at columns of interest, we can insert them in double square brackets.

In [9]:
aita_submissions[['author', 'date', 'title', 'selftext', 'url', 'subreddit', 'score', 'num_comments', 'num_crossposts']]

author                      date  \
0     Additional-Pizza-805 2020-07-24 19:13:49+00:00   
1              decadel8ter 2020-07-24 14:37:13+00:00   
2               Snoo_66130 2020-07-24 12:35:35+00:00   
3      ohnoihaveabluechair 2020-07-24 10:56:56+00:00   
4           FormalLettuce3 2020-07-24 10:52:08+00:00   
...                    ...                       ...   
2927   BackgroundJellyfish 2018-08-31 21:39:49+00:00   
2928       Marylebone_Road 2018-08-30 17:00:31+00:00   
2929         treefiddyfive 2018-08-26 09:33:45+00:00   
2930          grizzythekid 2018-06-19 02:07:05+00:00   
2931         Pettheftthrow 2018-05-11 03:28:07+00:00   

                                                                                                                                               title  \
0                                                                                   AITA for kicking my cousin off of my sister’s wedding Zoom call?   
1                                                                        AITA for resenting my family for something that happened over a decade ago?   
2                                                                                          AITA for telling my step dad to stop trying to be my dad?   
3                                                                               AITA for confronting my SIL for wearing clothes that belonged to me?   
4                                                     AITA for saying we'd only help with my ex's kid's party if we could tell people we're engaged?   
...                                                                                                                                              ...   
2927                                                                                AITA for hitting my girlfriend out of reflex for her scaring me?   
2928  AITA for thinking that this sub is only so people can have their decisions validated and never actually post something assholish they've done?   
2929                                                                                               AITA for not believing my daughter is non-binary?   
2930                                                                           AITA for throwing a soda on the ground near the dude I bought it for?   
2931                                                                                                         AITA for refusing to return a lost pet?   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 selftext  \
0     My [27M] older sister [30F] and her fiancé [31M] were planning for over a year for their wedding to be this month. Obviously, they can’t have the wedding as planned, but they still would like to get married, so they decided on a “Zoom” wedding where all of the family/friends would just call in to watch the officiant, my sister, and her fiancé. \n\nMy sister didn’t want to be in charge of hosting the Zoom call because she thought it would stress her out, so she asked me to and I gladly accept...   
1     when i was 15 i was in a car accident. i was riding my bike on new bike lanes that my city had installed, and a car hit me. I ended up having to go to the hospital, nothing major, the car was turning onto the main road when it hit me so it was going well below the speed limit. but since it was a car accident, i was forced into the ambulance and shipped off to the hospital to get x-rays.\n\nsince i was a minor, I wasn't allowed to be released from the hospital without my family coming to pick..

## Collect Reddit Posts (By Keyword)

To search by a keyword, we will add `q=` and insert a query phrase, the rapper "Missy Elliott."

In [10]:
api_request_generator = api.search_submissions(q='Missy Elliott', score = ">2000")

Once this generator is set up, we can use it to collect Reddit posts. The code below is a list comprehension that loops through the generator and extracts relevant data for each matching Reddit post. It then turns that list into a Pandas DataFrame.

In [11]:
missy_submissions = pd.DataFrame([submission.d_ for submission in api_request_generator])

*The cell above should take a while to run. It's searching through Reddit's entire history. It's ok if you periodically get errors while it's running.*

To sort by and manipulate date information, let's transform the date into datetime values.

In [12]:
missy_submissions['date'] = pd.to_datetime(missy_submissions['created_utc'], utc=True, unit='s')

To only look at columns of interest, we can insert them in double square brackets.

In [13]:
missy_submissions[['author', 'date', 'title', 'selftext', 'url', 'subreddit', 'score', 'num_comments', 'num_crossposts', ]]

author                      date  \
0             Origai 2020-04-12 17:21:30+00:00   
1        Chriscftb97 2019-10-04 20:21:19+00:00   
2        Chriscftb97 2019-09-14 16:29:34+00:00   
3        Chriscftb97 2019-08-30 21:00:33+00:00   
4        hugh__honey 2019-08-22 18:12:01+00:00   
5           errgreen 2019-07-26 19:59:15+00:00   
6       emitremmus27 2019-06-14 13:59:53+00:00   
7          DanWofSoc 2018-05-04 15:59:44+00:00   
8         rejeremiad 2018-01-22 17:02:21+00:00   
9       silverwolfer 2017-10-23 22:34:16+00:00   
10            hennny 2017-01-30 11:40:38+00:00   
11        BigLeJaffe 2016-04-26 20:33:14+00:00   
12       TheHHHRobot 2016-04-21 17:50:05+00:00   
13           RaHxRaH 2015-11-12 14:05:16+00:00   
14            caphis 2015-05-28 02:12:13+00:00   
15  All_Under_Heaven 2015-02-02 01:27:53+00:00   

                                                                                                                                                                                                                                                                        title  \
0                                                                                                                                                                                                                                          When Missy Elliott is a Heidi Stan   
1                                                                                               DaBaby's "KIRK" sells 147K First Week (8K Pure). Kevin Gates' "I'm Him" sells 71K First Week (15K Pure). Young M.A's "Herstory in the Making" sells 22K First Week (5K Pure).   
2                                                                                                                                            Post Malone's "Hollywood's Bleeding" sells 493K First Week (210K Pure). EARTHGANG's "Mirrorland" sells 12K First Week (2K Pure).   
3   BROCKHAMPTON's "GINGER" sells 77K First Week (55K Pure). Jeezy's "TM104: The Legend of the Snowman" sells 50K First Week (22K Pure). Missy Elliott's "ICONOLOGY" sells 17K First Week (11K Pure). SAINt JHN's "Ghetto Lenny's Love Songs" sells 13K First Week (4K Pure).   
4                                                                                                                                                                                          Missy Elliott announces first new album in 14 years, ICONOLOGY, coming out TONIGHT   
5                                                                                                                                                                                                                  Lizzo - Tempo (feat. Missy Elliott) [Official Music Video]   
6                                                                                                                                                                                    Missy Elliott becomes first female hip-hop artist inducted into Songwriters Hall of Fame   
7                                                                                                                        Friday Midday MAGAthread! - 05/04/2018 - Focus on the Midterms Volume 1: House Seats : CA-10 Jeff Denham, CA-25 Steve Knight, CA-48 Dana Rohrabacher   
8                                                                                                                                                      Malia Obama's Brand New Car is Disgusting | It isn't, refers to "Limosine One" used by the POTUS, saved you 257 clicks   
9                                                                                                                                                                                                                                Missy Elliott doing you a /r/personalfinance   
10                                                                                                                                                              

To look at the subreddits where "Missy Elliott" appears the most often, we can use the `.value_counts()` method.

In [77]:
missy_submissions['subreddit'].value_counts()

hiphopheads           7
Music                 2
The_Donald            2
Showerthoughts        1
rupaulsdragrace       1
savedyouaclick        1
reactiongifs          1
BlackPeopleTwitter    1
Name: subreddit, dtype: int64

## Collect Reddit Comments (By Keyword)

To collect Reddit *comments* rather than posts, we can use `api.search_comments()` rather than `api.search_submissions()`.

In [39]:
api_request_generator = api.search_comments(q='Missy Elliott', score = ">2000")

In [40]:
missy_comments = pd.DataFrame([comment.d_ for comment in api_request_generator])

## Collect Reddit Posts and Comments (By Multiple Keywords)

To search for multiple phrases in posts —  such as posts that mention the author George Orwell OR the author J.R.R. Tolkein — we can use parentheses and the bitwise OR operator `|`

In [56]:
api_request_generator = api.search_comments(q='(George Orwell)|(J. R. R. Tolkien)')

To search for multiple phrases in posts —  such as posts that mention Shakespeare AND Beyonce — we can use parentheses and the bitwise AND operator `&`

In [56]:
api_request_generator = api.search_comments(q='(Shakespeare)&(Beyonce)')

## Collect Reddit Posts and Comments (By Date Range)

In [73]:
import datetime as dt

start_epoch=int(dt.datetime(2020, 1, 1).timestamp())
end_epoch=int(dt.datetime(2020, 1, 10).timestamp())

api_request_generator = api.search_comments(q='(Shakespeare)&(Beyonce)"', after = start_epoch, before=end_epoch)